<a href="https://colab.research.google.com/github/JanhaviSoni7/Deep-Learning-Projects/blob/main/Plant_Disease_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set seeds for reproducibility


In [ ]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

Importing the dependencies


In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

data curation

In [ ]:
!pip install kaggle

In [ ]:
kaggle_credentails=json.load(open("/content/kaggle (2).json"))

In [ ]:
#setup kaggle api key as environment variables
os.environ['KAGGLE_USERNAME']=kaggle_credentails["username"]
os.environ['KAGGLE_KEY']=kaggle_credentails["key"]

In [ ]:
os.system('kaggle datasets download -d jawadali1045/20k-multi-class-crop-disease-images')

In [ ]:
!ls


In [ ]:
#unzip the downloaded dataset
with ZipFile("/content/20k-multi-class-crop-disease-images.zip","r") as zip_ref:
  zip_ref.extractall()

In [ ]:
%cd /content
!pwd


In [ ]:
!ls


In [ ]:
import os
import zipfile

# Path to the zip file
zip_file_path = "20k-multi-class-crop-disease-images.zip"

# Extract the zip file to a directory with the same name as the zip file (without extension)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("20k-multi-class-crop-disease-images")

# List the contents of the extracted directory
extracted_dir = "20k-multi-class-crop-disease-images"
print("Contents of extracted directory:", os.listdir(extracted_dir))


In [ ]:
import os

# List the contents of the dataset directory
dataset_dir = "20k-multi-class-crop-disease-images"
print("Dataset directory contents:", os.listdir(dataset_dir))

# Paths to the Train and Validation directories
train_dir = os.path.join(dataset_dir, "Train")
validation_dir = os.path.join(dataset_dir, "Validation")

# List the contents of the Train directory
print("Train directory contents:", os.listdir(train_dir))

# List the contents of the Validation directory
print("Validation directory contents:", os.listdir(validation_dir))

# Example: List and inspect the first 5 images in the first class of the Train directory
first_class_train = os.listdir(train_dir)[0]  # Pick the first class/category in Train
first_class_train_path = os.path.join(train_dir, first_class_train)

print(f"Contents of {first_class_train} in Train directory:", os.listdir(first_class_train_path)[:5])
print(f"Number of images in {first_class_train}:", len(os.listdir(first_class_train_path)))

# Similarly, inspect the first 5 images in the first class of the Validation directory
first_class_val = os.listdir(validation_dir)[0]  # Pick the first class/category in Validation
first_class_val_path = os.path.join(validation_dir, first_class_val)

print(f"Contents of {first_class_val} in Validation directory:", os.listdir(first_class_val_path)[:5])
print(f"Number of images in {first_class_val}:", len(os.listdir(first_class_val_path)))


Number of Classes=38


In [ ]:


train_dir = "20k-multi-class-crop-disease-images/Train"

# List all class directories in the Train folder
if os.path.exists(train_dir):
    class_names = os.listdir(train_dir)
    print("Available classes in Train directory:", class_names)
else:
    print(f"Directory {train_dir} does not exist.")


Data preprocessing

In [ ]:
#Dataset path
base_dir='/content/Train'

In [ ]:
image_path='/content/Train/Army worm/11.jpg'
#read the image
img= mpimg.imread(image_path)

print(img.shape)
#display the image
plt.imshow(img)
plt.axis('off') #turn off axis numbers
plt.show()

In [ ]:
#Image Parameters
img_size=224
batch_size=32

Train and split

In [ ]:
#image data generator
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

In [ ]:
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)


In [ ]:
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)


Convolutional Neural Network

In [ ]:
#Model Definition
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_size,img_size,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(train_generator.num_classes,activation='softmax'))


In [ ]:
#model summary
model.summary()

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

**Model Training**

In [ ]:
pip install --upgrade tensorflow


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


**Model Evaluation**

In [ ]:
# Model Evaluation
print("Evaluating model....")
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
#plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Test'], loc='upper left')
plt.show()


#plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train','Test'],loc='upper left')
plt.show()

**Building a Predictive System**

In [ ]:
from PIL import Image
import numpy as np

# Function to load and preprocess the image using Pillow
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    img = Image.open(image_path)
    # Resize the image
    img = img.resize(target_size)
    # Convert image to numpy array
    img_array = np.array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Scale the image values to [0, 1]
    img_array = img_array.astype('float32') / 255.
    return img_array

# Function to predict the class of an image
def predict_image_class(model, image_path, class_indices):
    # Load and preprocess the image
    preprocessed_img = load_and_preprocess_image(image_path)
    # Get model predictions
    predictions = model.predict(preprocessed_img)
    # Find the index of the highest prediction score
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    # Map the index to the class name
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name


In [ ]:
#create a mapping from class indices to class names
class_indices={
    v:k for k,v in train_generator.class_indices.items()
}

In [ ]:
class_indices

In [ ]:
with open("class_indices.json", "w") as json_file:
    json.dump(class_indices, json_file)

In [ ]:
#image_path = '/content/test_apple_black_rot.jpg'
image_path = '/content/corn_northen_leaf_blight_test.jpg'
predicted_class_name = predict_image_class(model, image_path, class_indices)
print("Predicted Class name: ", predicted_class_name)

**Save the model to google drive**

In [ ]:
!ls /content/drive/My\ Drive/


In [ ]:
model.save('/content/drive/My Drive/model_name.h5')
